In [ ]:
from pathlib import Path
from plotting import plot_comparison, plot_image, plot_label

![alt text](./ps_tasks.png)

# Step 0: Setup workflow Tracing

- Running the following cell will make sure that that the current workflow is "empty" and ready to be used.

In [4]:
from plantseg.tasks.workflow_handler import workflow_handler

workflow_handler.clean_dag()

print(f"Run-Inputs: {workflow_handler.dag.inputs}")
print(f"Tasks: {workflow_handler.dag.list_tasks}")

Run-Inputs: [{}]
Tasks: []


# Step 1: Import the input data

- First, we need to import the input data. This is done by running the following cell.
- Try to run the following cells with diffent inputs from the given test data (eg. [ovule_2d](https://drive.google.com/file/d/1Mfg3q-5Rj_oxLPUaqyTiqvVk9uKRB8xa/view?usp=sharing))
- Be careful that some parementers like the `stack_layout` might need to be adjusted based on the input data.

In [5]:
from plantseg.tasks.io_tasks import import_image_task

path = Path("PATH to your File")

ps_image = import_image_task(input_path=path, semantic_type='raw', stack_layout="YX", key="raw")

plot_image(ps_image.get_data(), title="Raw Image")

print(f"Run-Inputs: {workflow_handler.dag.inputs}")
print(f"Tasks: {workflow_handler.dag.list_tasks}")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/locerr/Projects/plantseg-workshop-psc-2025/PATH to your File'

# Step 2: Run a basic workflow

In [ ]:
from plantseg.io.voxelsize import VoxelSize
from plantseg.tasks.dataprocessing_tasks import gaussian_smoothing_task, image_rescale_to_voxel_size_task, set_biggest_instance_to_zero_task
from plantseg.tasks.prediction_tasks import unet_prediction_task
from plantseg.tasks.segmentation_tasks import dt_watershed_task, clustering_segmentation_task


smooth_image = gaussian_smoothing_task(image=ps_image, sigma=6.0)

plot_comparison(ps_image.get_data(), smooth_image.get_data(), title1="Raw Image", title2="Smooth Image")


# Step 3: Export image and save the workflow

- Runnning the following cell will export the image of the workflow and save the workflow file in the current directory.

In [ ]:
from plantseg.tasks.io_tasks import export_image_task

export_image_task(image=smooth_image, export_directory="./", name_pattern='{file_name}_export')

workflow_handler.save_to_yaml(path="./base_workflow.yaml")

print(f"Run-Inputs: {workflow_handler.dag.inputs}")
print("Tasks: ")
for task in workflow_handler.dag.list_tasks:
    print(f'- {task}')


# Step 4 Run the workflow from the command line

- Run the following command in the terminal to run the workflow from the command line.
```bash
plantseg --config base_workflow.yaml
```

# Step 5: Create a more complex workflow

- You can create a more complex workflow by adding more steps to the workflow.
- Try for example to use the `unet_prediction_tak` and the `dt_watershed_task` to segment one of the test images.
- To check the possible kewords arguments for each task you can see our [api documentation](https://kreshuklab.github.io/plant-seg/chapters/python_api/tasks/io_tasks/)

In [ ]:
from plantseg.io.voxelsize import VoxelSize
from plantseg.tasks.dataprocessing_tasks import gaussian_smoothing_task, image_rescale_to_voxel_size_task, set_biggest_instance_to_zero_task
from plantseg.tasks.prediction_tasks import unet_prediction_task
from plantseg.tasks.segmentation_tasks import dt_watershed_task, clustering_segmentation_task


In [ ]:
from plantseg.tasks.workflow_handler import workflow_handler

workflow_handler.clean_dag()

print(f"Run-Inputs: {workflow_handler.dag.inputs}")
print(f"Tasks: {workflow_handler.dag.list_tasks}")


##############################################
#
# Place your code here
#
##############################################


workflow_handler.save_to_yaml(path="./full_workflow.yaml")

print(f"Run-Inputs: {workflow_handler.dag.inputs}")
print("Tasks: ")
for task in workflow_handler.dag.list_tasks:
    print(f'- {task}')

# Step 6: Use plantseg functionals to interact with other libraries

- You can use the plantseg functionals to interact with other libraries like napari, scikit-image, etc.
- Plantseg functionals simply work with numpy arrays
- Replace the `random_image` with an image from the test data and try to use the functionals to segment the image.

In [ ]:
import napari 
# Start a napari viewer
viewer = napari.Viewer()

In [20]:
from plantseg.functionals.prediction import unet_prediction
from plantseg.functionals.dataprocessing import normalize_01, image_gaussian_smoothing, image_rescale
from plantseg.functionals.segmentation import dt_watershed, gasp, multicut, mutex_ws
from plantseg.io import smart_load
from pathlib import Path
import numpy as np

random_image = np.random.rand(512, 512)
# test_image = smart_load(Path("Path to your image"))

prediction = unet_prediction(raw=random_image, 
                             input_layout="YX",
                             model_name="confocal_2D_unet_ovules_ds2x",
                             # patch=(1, 64, 64),
                             #patch_halo=(0, 0, 0),
                             model_id=None, device="mps")

# you can play with the image contrast and you can see the effect of
# patch and patch_halo on the result
viewer.add_image(prediction)

INFO: P [MainThread] 2025-03-12 09:54:10,916 plantseg.functionals.prediction.prediction - Zoo prediction: Running model from PlantSeg official zoo.
INFO: P [MainThread] 2025-03-12 09:54:10,958 plantseg.functionals.prediction.prediction - Computing theoretical minimum halo from model.
INFO: P [MainThread] 2025-03-12 09:54:21,944 plantseg.functionals.prediction.prediction - For raw in shape (950, 1100): set patch shape (1, 950, 1100), set halo shape (0, 0, 0)
INFO: P [MainThread] 2025-03-12 09:54:24,276 plantseg.functionals.prediction.utils.array_predictor - Using batch size of 1 for prediction


100%|██████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.74s/it]


<Image layer 'prediction [1]' at 0x392178050>